#### 신경망 - Neural networks

torch.nn 패키지를 사용하여 신경망을 구축할 수 있습니다. 

지금까지 autograd를 살펴봤는데, nn은 모델을 정의하고 미분하는데 바로 이 autograd를 사용합니다.

nn.Module은 계층과 output을 반환하는 forward(input), 계층들을 포함하고 있습니다.

신경망의 학습절차는 다음과 같습니다.

- 학습 가능한 매개변수(Weight)를 갖고 있는 신경망을 정의합니다.

- 데이터셋의 입력을 반복합니다.

- 입력을 신경망에서 전파 합니다.

- 손실을 계산합니다. (출력값이 정답으로부터 얼마나 멀리 떨어져 있는지)

- 신경망의 매개변수에 변화도를 역으로 전파합니다. 

- 신경망의 가중치를 갱신합니다. 일반적으로 간단한 갱신 규칙을 사용합니다. (weight = weight-learning_rate*gradient)

#### 신경망 정의하기

In [1]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F 

In [64]:
class Net(nn.Module):

    #생성자
    def __init__(self):
        super(Net, self).__init__()
        #입력 이미지 채널 1개, 출력 채녈 6개, 3 * 3의 정사각 컨볼루션 행렬
        self.conv1 = nn.Conv2d(3,6,(3,3))
        self.conv2 = nn.Conv2d(6,16,(3,3))

        # 어파인 연산 : y = Wx + b
        self.fc1 = nn.Linear(16*6*6,120) #6 * 6은 이미지 차원
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
    
    def forward(self,x):
        # (2 * 2) 크기 윈도우에 대해 max pooling
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        print(x.shape)
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        print(x.shape)
        x = x.view(-1,self.num_flat_features(x))
        print(x.shape)
        x = F.relu(self.fc1(x))
        print(x.shape)
        x = F.relu(self.fc2(x))
        print(x.shape)
        x = self.fc3(x)
        return x 
    
    def num_flat_features(self,x):
        size = x.size()[1:] #배치 차원을 제외한 모든 차원에 대하여 
        num_features = 1 
        for s in size:
            num_features *= s 
        return num_features 


In [65]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


forward 함수를 정의하고 나면 autograd를 사용하여 저절로 backword 함수를 정의합니다.

forward 함수에서 어떠한 Tensor 연산을 사용할 수 있습니다.


모델의 학습 가능한 weights는 net.parameters()에 의해 반환됩니다.

In [66]:
params = net.parameters()
params = list(params)

print(params[0].size())
print(params[1].size())
print(params[9].size())


torch.Size([6, 3, 3, 3])
torch.Size([6])
torch.Size([10])


임의의 32 * 32 입력값을 해보겠습니다. 

이 신경망의 예상 입력 사이즈는 32 32입니다. 

In [69]:
inputs = torch.randn([1,3,32,32])


In [68]:
out = net(inputs)
print(out)

torch.Size([1, 6, 15, 15])
torch.Size([1, 16, 6, 6])
torch.Size([1, 576])
torch.Size([1, 120])
torch.Size([1, 84])
tensor([[ 0.0919, -0.0190, -0.0535,  0.1153,  0.1043, -0.0832,  0.0660,  0.0445,
          0.0120, -0.0402]], grad_fn=<AddmmBackward>)


In [58]:
net.zero_grad()
out.backward(torch.randn(1,10))

In [59]:
output = net(inputs)
target = torch.randn(10)

target = target.view(1,-1)

criterion = nn.MSELoss()
print('target :',target)
print('output :',output)
loss = criterion(output,target)
print(loss)

torch.Size([1, 6, 15, 15])
torch.Size([1, 16, 6, 6])
torch.Size([1, 576])
torch.Size([1, 120])
torch.Size([1, 84])
target : tensor([[-0.5936,  0.8635,  0.7650, -0.5059, -0.4012,  0.6895, -1.5084, -2.5587,
          0.7649,  0.0465]])
output : tensor([[ 0.1347, -0.0012, -0.0139,  0.0999,  0.0963,  0.0687, -0.1841, -0.0972,
          0.0418, -0.0691]], grad_fn=<AddmmBackward>)
tensor(1.1234, grad_fn=<MseLossBackward>)


In [47]:
print(loss.grad_fn)

In [48]:
print(loss.grad_fn.next_functions[0][0])

In [49]:
print((loss.grad_fn.next_functions[0][0].next_functions[0][0]))

역전파 

오차를 역전파하기 위해 해야할 것 : loss.backward()입니다. 

존재하는 변화도를 없애는 것이 필요한데, 그렇지 않으면 변화도는 기존의 변화도에 누적될 것입니다.